# Advanced Usage

This notebook covers advanced patterns and customization options using real climate datasets.

In [ ]:
import xarray as xr

from xarray_plotly import config, xpx

config.notebook()  # Configure Plotly for notebook rendering

## Load Tutorial Datasets

In [ ]:
# NCEP air temperature: 3D (time, lat, lon)
ds_air = xr.tutorial.open_dataset("air_temperature")
air = ds_air["air"]

# ERA-Interim: 4D (month, level, latitude, longitude)
ds_era = xr.tutorial.open_dataset("eraint_uvz")

print(f"Air temperature: {air.attrs.get('long_name', air.name)}")
print(f"  Dimensions: {dict(air.sizes)}")
print(f"  Units: {air.attrs.get('units', 'N/A')}")

## Working with xarray Attributes

xarray_plotly automatically uses metadata from xarray attributes for labels:

In [ ]:
# The air temperature DataArray already has rich metadata
print(f"Variable attrs: {air.attrs}")
print(f"Time coord attrs: {air.coords['time'].attrs}")

In [ ]:
# Labels are automatically extracted from attrs
fig = xpx(air.sel(lat=[40, 50, 60], lon=260, method="nearest")).line(
    title="Temperature with Auto-Labels from Metadata"
)
fig

### Configuring Label Behavior

Use `config.set_options()` to control how labels are extracted from attributes:

In [ ]:
# Disable units in labels
with config.set_options(label_include_units=False):
    fig = xpx(air.sel(lat=[40, 50, 60], lon=260, method="nearest")).line(
        title="Without Units in Labels"
    )
fig

### Overriding Labels

You can override the automatic labels:

In [ ]:
fig = xpx(air.sel(lat=[40, 50, 60], lon=260, method="nearest")).line(
    labels={
        "air": "Temperature (K)",
        "time": "Date",
        "lat": "Latitude",
    },
    title="Custom Labels",
)
fig

## Advanced Dimension Assignment

### Complex Slot Assignments with 4D Data

The ERA-Interim dataset has 4 dimensions - perfect for demonstrating complex assignments:

In [ ]:
# U-wind: (month, level, latitude, longitude)
u = ds_era["u"]
print(f"U-wind dimensions: {dict(u.sizes)}")
print(f"Pressure levels: {u.coords['level'].values} hPa")

In [ ]:
# Zonal mean profile: use line_dash for pressure level, color for month
u_zonal = u.mean("longitude")

fig = xpx(u_zonal).line(
    x="latitude",
    color="month",
    line_dash="level",
    title="Zonal Mean U-Wind: Multiple Visual Encodings",
)
fig

### Reducing Dimensions Before Plotting

When you have more dimensions than slots, reduce them first:

In [ ]:
# Average over months to compare pressure levels
fig = xpx(u.mean(["month", "longitude"])).line(
    x="latitude",
    color="level",
    title="Mean Zonal Wind Profile by Pressure Level",
)
fig

In [ ]:
# Select specific slices for focused analysis
fig = xpx(u.sel(level=200)).imshow(
    facet_col="month",
    color_continuous_scale="RdBu_r",
    color_continuous_midpoint=0,
    title="200 hPa U-Wind (Jet Stream Level)",
)
fig

## Custom Styling

### Themes

In [ ]:
fig = xpx(air.sel(lat=45, lon=260, method="nearest")).line(
    template="plotly_dark",
    title="Dark Theme",
)
fig

### Custom Colors

In [ ]:
import plotly.express as px

fig = xpx(air.sel(lat=[30, 45, 60], lon=260, method="nearest")).line(
    color_discrete_sequence=px.colors.qualitative.Set2,
    title="Custom Color Palette",
)
fig

### Heatmap Colorscales

Climate data often benefits from diverging colorscales centered at meaningful values:

In [ ]:
# Temperature anomaly from mean
air_mean = air.mean("time")
air_anomaly = air.isel(time=0) - air_mean
air_anomaly.name = "temperature_anomaly"
air_anomaly.attrs["long_name"] = "Temperature Anomaly"
air_anomaly.attrs["units"] = "K"

# Diverging colorscale centered at zero
fig = xpx(air_anomaly).imshow(
    color_continuous_scale="RdBu_r",
    color_continuous_midpoint=0,
    title="Temperature Anomaly (Diverging Colorscale)",
)
fig

## Post-Creation Customization

All plots return Plotly `Figure` objects that can be extensively customized:

In [ ]:
fig = xpx(air.sel(lat=[30, 45, 60], lon=260, method="nearest")).line()

# Add horizontal reference line (freezing point in Kelvin)
fig.add_hline(y=273.15, line_dash="dash", line_color="gray", annotation_text="Freezing Point")

# Update layout
fig.update_layout(
    title="Temperature with Freezing Point Reference",
    legend={
        "orientation": "h",
        "yanchor": "bottom",
        "y": 1.02,
        "xanchor": "right",
        "x": 1,
    },
)

fig

### Modifying Traces

In [ ]:
fig = xpx(air.sel(lat=[30, 45, 60], lon=260, method="nearest").isel(time=slice(0, 100))).line()

# Make all lines thicker
fig.update_traces(line_width=3)

fig.update_layout(title="Thicker Lines")
fig

## Exporting Figures

### Interactive HTML

```python
fig.write_html("interactive_plot.html")
```

### Static Images

Requires `kaleido`: `pip install kaleido`

```python
fig.write_image("plot.png", scale=2)  # High resolution
fig.write_image("plot.svg")  # Vector format
fig.write_image("plot.pdf")  # PDF
```

## Integration with xarray Operations

xarray_plotly works seamlessly with xarray's computation methods:

### Rolling Mean

In [ ]:
# 30-day rolling mean to smooth daily variations
air_smooth = air.sel(lat=45, lon=260, method="nearest").rolling(time=30, center=True).mean()

fig = xpx(air_smooth).line(
    title="30-Day Rolling Mean Temperature",
)
fig

### Groupby Operations (Climatology)

In [ ]:
# Monthly climatology
monthly_clim = air.sel(lon=260, method="nearest").groupby("time.month").mean()

fig = xpx(monthly_clim.sel(lat=[30, 45, 60], method="nearest")).line(
    title="Monthly Temperature Climatology",
)
fig.update_xaxes(
    tickmode="array",
    tickvals=list(range(1, 13)),
    ticktext=["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
)
fig

### Resampling

In [ ]:
# Weekly mean temperature
weekly = air.sel(lat=45, lon=260, method="nearest").resample(time="W").mean()

fig = xpx(weekly).line(
    title="Weekly Mean Temperature",
    markers=True,
)
fig

### Spatial Statistics

In [ ]:
# Zonal mean temperature over time
zonal_mean = air.mean("lon")

# Show as heatmap (Hovmoller diagram)
fig = xpx(zonal_mean).imshow(
    title="Zonal Mean Temperature (Hovmoller Diagram)",
    color_continuous_scale="thermal",
)
fig